In [99]:
%pwd  #look at the current work dir

'/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-ml-pipeline'

In [100]:
%cd '/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-ml-pipeline'

/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-ml-pipeline


/Users/yannickflores/Library/Caches/pypoetry/virtualenvs/foreshadwownba-dvc-ml-pipeline-kQRXfENR-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [101]:
%pwd  #look at the current work dir

'/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-ml-pipeline'

In [102]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [103]:
df = pd.read_csv("./data/input/nba_training_dataset.csv")

df = df.sort_values(
        ['id_season','tm', 'game_nb']
    )

In [104]:
df.dtypes

id_season                                int64
id                                      object
game_nb                                  int64
extdom                                  object
tm                                      object
opp                                     object
results                                  int64
pts_tm                                   int64
pts_opp                                  int64
before_average_pts_tm                  float64
before_average_lastfivegame_pts_tm     float64
before_average_lasttengame_pts_tm      float64
diff_all_minus_lastfivegame_pts_tm     float64
diff_all_minus_lasttengame_pts_tm      float64
before_average_pts_opp                 float64
before_average_lastfivegame_pts_opp    float64
before_average_lasttengame_pts_opp     float64
diff_all_minus_lastfivegame_pts_opp    float64
diff_all_minus_lasttengame_pts_opp     float64
before_average_W_ratio                 float64
before_average_lastfivegame_W_ratio    float64
before_averag

In [105]:
df.head()

,id_season,id,game_nb,extdom,tm,opp,results,pts_tm,pts_opp,before_average_pts_tm,before_average_lastfivegame_pts_tm,before_average_lasttengame_pts_tm,diff_all_minus_lastfivegame_pts_tm,diff_all_minus_lasttengame_pts_tm,before_average_pts_opp,before_average_lastfivegame_pts_opp,before_average_lasttengame_pts_opp,diff_all_minus_lastfivegame_pts_opp,diff_all_minus_lasttengame_pts_opp,before_average_W_ratio,before_average_lastfivegame_W_ratio,before_average_lasttengame_W_ratio,diff_all_minus_lastfivegame_W_ratio,diff_all_minus_lasttengame_W_ratio,before_season_ratio,rest,before_ratio_win_ext,before_ratio_win_dom,last_game_overtime,before_streak_w_l
0,2011,2010-10-29_ATL_PHI,2,ext,ATL,PHI,1,104,101,119.0,119.0,119.0,0.0,0.0,104.0,104.0,104.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,2.0,1.0,0.0,NOT,1.0
1,2011,2010-10-30_WAS_ATL,3,dom,ATL,WAS,1,99,95,111.5,111.5,111.5,0.0,0.0,102.5,102.5,102.5,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,1.0,1.0,0.0,NOT,2.0
2,2011,2010-11-02_ATL_CLE,4,ext,ATL,CLE,1,100,88,107.3,107.3,107.3,0.0,0.0,100.0,100.0,100.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,3.0,1.0,1.0,NOT,3.0
3,2011,2010-11-03_DET_ATL,5,dom,ATL,DET,1,94,85,105.5,105.5,105.5,0.0,0.0,97.0,97.0,97.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,1.0,1.0,1.0,NOT,4.0
4,2011,2010-11-05_ATL_MIN,6,ext,ATL,MIN,1,113,103,103.2,103.2,103.2,0.0,0.0,94.6,94.6,94.6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,2.0,1.0,1.0,NOT,5.0


In [106]:
df.drop([
        'results', 
        'id',
        'id_season',	
        # 'game_nb',	
        # 'extdom',	
        'tm',	
        'opp',
        'pts_tm',	
        'pts_opp'
        ], axis=1).head()

,game_nb,extdom,before_average_pts_tm,before_average_lastfivegame_pts_tm,before_average_lasttengame_pts_tm,diff_all_minus_lastfivegame_pts_tm,diff_all_minus_lasttengame_pts_tm,before_average_pts_opp,before_average_lastfivegame_pts_opp,before_average_lasttengame_pts_opp,diff_all_minus_lastfivegame_pts_opp,diff_all_minus_lasttengame_pts_opp,before_average_W_ratio,before_average_lastfivegame_W_ratio,before_average_lasttengame_W_ratio,diff_all_minus_lastfivegame_W_ratio,diff_all_minus_lasttengame_W_ratio,before_season_ratio,rest,before_ratio_win_ext,before_ratio_win_dom,last_game_overtime,before_streak_w_l
0,2,ext,119.0,119.0,119.0,0.0,0.0,104.0,104.0,104.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,2.0,1.0,0.0,NOT,1.0
1,3,dom,111.5,111.5,111.5,0.0,0.0,102.5,102.5,102.5,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,1.0,1.0,0.0,NOT,2.0
2,4,ext,107.3,107.3,107.3,0.0,0.0,100.0,100.0,100.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,3.0,1.0,1.0,NOT,3.0
3,5,dom,105.5,105.5,105.5,0.0,0.0,97.0,97.0,97.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,1.0,1.0,1.0,NOT,4.0
4,6,ext,103.2,103.2,103.2,0.0,0.0,94.6,94.6,94.6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,2.0,1.0,1.0,NOT,5.0


In [91]:
df[df['id']=='2010-10-29_ATL_PHI']

,id_season,id,game_nb,extdom,tm,opp,results,pts_tm,pts_opp,before_average_pts_tm,before_average_lastfivegame_pts_tm,before_average_lasttengame_pts_tm,diff_all_minus_lastfivegame_pts_tm,diff_all_minus_lasttengame_pts_tm,before_average_pts_opp,before_average_lastfivegame_pts_opp,before_average_lasttengame_pts_opp,diff_all_minus_lastfivegame_pts_opp,diff_all_minus_lasttengame_pts_opp,before_average_W_ratio,before_average_lastfivegame_W_ratio,before_average_lasttengame_W_ratio,diff_all_minus_lastfivegame_W_ratio,diff_all_minus_lasttengame_W_ratio,before_season_ratio,rest,before_ratio_win_ext,before_ratio_win_dom,last_game_overtime,before_streak_w_l
0,2011,2010-10-29_ATL_PHI,2,ext,ATL,PHI,1,104,101,119.0,119.0,119.0,0.0,0.0,104.0,104.0,104.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.646341,2.0,1.0,0.0,NOT,1.0
1855,2011,2010-10-29_ATL_PHI,2,dom,PHI,ATL,0,101,104,87.0,87.0,87.0,0.0,0.0,97.0,97.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.329268,2.0,0.0,0.0,NOT,-1.0


In [92]:

df_joined = pd.merge(
    df[['id', 'before_average_W_ratio', 'tm', 'opp', 'results']],
    df[['id', 'before_average_W_ratio', 'tm', 'opp']],
    how='left',
    on='id')

# Remove duplciate - comditon is where tm_x != tm_y

df_joined = df_joined[df_joined['tm_x'] != df_joined['tm_y']]

df_joined[df_joined['id']=='2010-10-29_ATL_PHI']

,id,before_average_W_ratio_x,tm_x,opp_x,results,before_average_W_ratio_y,tm_y,opp_y
1,2010-10-29_ATL_PHI,1.0,ATL,PHI,1,0.0,PHI,ATL
3561,2010-10-29_ATL_PHI,0.0,PHI,ATL,0,1.0,ATL,PHI
